In [ ]:
import pandas as pd
from dotenv import dotenv_values
import psycopg2

import numpy as np

In [10]:
env_vars = dotenv_values('<postgres.env_path>')

DATABASE_NAME = env_vars.get('DATABASE_NAME')
DATABASE_USER = env_vars.get('DATABASE_USER')
DATABASE_PASSWORD = env_vars.get('DATABASE_PASSWORD')
DATABASE_HOST = env_vars.get('DATABASE_HOST')
OPENAI_API_KEY = env_vars.get('OPENAI_API_KEY')

In [3]:
df = pd.read_excel('./Postre_Data/Siparisler_Tabular.xlsx') # We deleted this file due to the privacy

In [4]:
df2 = pd.read_excel('./Postre_Data/Gunluk_Uretim_Adetleri_Raporu_Tabular.xlsx') # We deleted this file due to the privacy

In [5]:
def row_to_string(row):
    return ', '.join([f"{col}: {row[col]}" for col in row.index])

df['text'] = df.apply(row_to_string, axis=1)

In [6]:
df = df[:1000]

In [7]:
from openai import OpenAI
client = OpenAI(api_key="<open_ai_api_key>")

In [11]:
def get_embedding(text):
    return client.embeddings.create(input = [text], model='text-embedding-3-large').data[0].embedding

df['embedding'] = df['text'].apply(get_embedding)


KeyboardInterrupt



In [ ]:
conn = psycopg2.connect(
    dbname=DATABASE_NAME,
    user=DATABASE_USER,
    password=DATABASE_PASSWORD,
    host=DATABASE_HOST
)

In [ ]:
cur = conn.cursor()

# Tablo oluşturma
cur.execute('''
CREATE TABLE IF NOT EXISTS siparis_embeddings (
    id SERIAL PRIMARY KEY,
    text TEXT,
    embedding VECTOR(3072)  -- OpenAI embedding boyutu
)
''')
conn.commit()

In [12]:
for index, row in df.iterrows():
    embedding_vector = np.array(row['embedding'])
    cur.execute(
        'INSERT INTO siparis_embeddings (text, embedding) VALUES (%s, %s)',
        (row['text'], embedding_vector.tolist())
    )

conn.commit()

In [13]:
df2 = df2[50:1050]

In [14]:
df2['text'] = df2.apply(row_to_string, axis=1)

In [15]:
df2['embedding'] = df2['text'].apply(get_embedding)

In [16]:
cur = conn.cursor()

# Tablo oluşturma
cur.execute('''
CREATE TABLE IF NOT EXISTS uretim_embeddings (
    id SERIAL PRIMARY KEY,
    text TEXT,
    embedding VECTOR(3072)  -- OpenAI embedding boyutu
)
''')
conn.commit()

In [17]:
for index, row in df2.iterrows():
    embedding_vector = np.array(row['embedding'])
    cur.execute(
        'INSERT INTO uretim_embeddings (text, embedding) VALUES (%s, %s)',
        (row['text'], embedding_vector.tolist())
    )

conn.commit()

In [18]:
cur=conn.cursor()
cur.execute("SELECT text, embedding FROM uretim_embeddings")
results = cur.fetchall()

texts = []
embeddings = []
for row in results:
    texts.append(row[0])
    embeddings.append(row[1])


In [20]:
import ast

In [21]:
def retrieve_and_generate(query_text, embeddings, texts, k=5):
    # Step 1: Retrieve top-k texts based on similarity to query_text
    print(query_text)
    query_embedding = np.array(get_embedding(query_text))
    query_embedding = query_embedding.reshape(1, -1)  # Ensure the embedding is ;
    similarities = []

    for emb in embeddings:
        # Ensure the embedding is converted from string to numpy array
        emb = np.array(ast.literal_eval(emb)).reshape(1, -1)
        similarity = np.dot(query_embedding, emb.T) / (np.linalg.norm(query_embedding) * np.linalg.norm(emb))
        similarities.append(similarity[0][0])

    # Get top-k indices
    top_k_indices = np.argsort(similarities)[-k:][::-1]

    # Retrieve top-k texts
    top_k_texts = [texts[idx] for idx in top_k_indices]

    # Step 2: Generate text using OpenAI GPT model based on the retrieved texts
    prompt = f"""Retrieved texts: 
    {np.array(top_k_texts).reshape(-1, 1)}\n
    Query: {query_text}\nGenerate:"""
        
    return prompt

# Example usage
query_text = "Dikim olan ve C&A ürünleri"
generated_text = retrieve_and_generate(query_text, embeddings, texts)
print("Generated text:", generated_text)

Dikim olan ve C&A ürünleri
Generated text: Retrieved texts: 
    [['Order No: PLM16686, Model Kodu: DSG08713-001, Model Adı: 22775, Müşteri: C&A Buying GmbH & Co. KG, Bölüm: nan, Tedarikçi: Numune Dikim Bandı - C&A / ING / RESERVED  / lCW / BRAVADO, Müşteri Order No: nan, Rpt: nan, Marka: C&A, Renk Kodu: 0352 Red, İşlem: Kesim, RSN: nan, Tarih: 44664.0, Adet: 3.0, Kumaş: nan, Kumaş Tipi: nan, Kalite Tipi: Sağlam']
 ['Order No: PLM24012, Model Kodu: DSG11710-001, Model Adı: YS-2779 B, Müşteri: C&A Buying GmbH & Co. KG, Bölüm: nan, Tedarikçi: Numune Dikim Bandı - C&A / ING / RESERVED  / lCW / BRAVADO, Müşteri Order No: nan, Rpt: nan, Marka: C&A, Renk Kodu: CUTTING 42, İşlem: Dikim, RSN: nan, Tarih: 44820.0, Adet: 3.0, Kumaş: nan, Kumaş Tipi: nan, Kalite Tipi: Sağlam']
 ['Order No: PLM16769, Model Kodu: DSG08759-001, Model Adı: CB0422-01, Müşteri: C&A Buying GmbH & Co. KG, Bölüm: nan, Tedarikçi: Numune Dikim Bandı - C&A / ING / RESERVED  / lCW / BRAVADO, Müşteri Order No: nan, Rpt: nan, M

In [23]:
cur=conn.cursor()
cur.execute("SELECT text, embedding FROM siparis_embeddings")
results_uretim = cur.fetchall()

texts_siparis = []
embeddings_siparis = []
for row in results_uretim:
    texts_siparis.append(row[0])
    embeddings_siparis.append(row[1])


In [24]:
query_text = "Departmanı Young Girl(8716) olan kayıtlar"
generated_text = retrieve_and_generate(query_text, embeddings_siparis, texts_siparis)
print("Generated text:", generated_text)

Departmanı Young Girl(8716) olan kayıtlar
Generated text: Retrieved texts: 
    [['Durum: Açık, Statü: Planlandı, Booking -> Order: EVET, Set Order: HAYIR, Set Parça Sayısı: nan, Order Tipi: Firm Order, Booking Type: Müşteri, Order Grubu: nan, Satış Tipi: Müşteri Koleksiyonu, Sertifikasyon: BCI, Order No: H&M241063, Order Geliş Tarihi: 45436, Sisteme Giriş Tarihi: 45436, Satır ya da Sütun M.İsteme Tarihi: 45488, YIL: 2024, AY: 7, HAFTA: 29, ÇEYREK: Q3, Müşteri Order No: 970190-8716, RPT: nan, Açıklama: nan, Model Kodu: H&M05772, Model Adı: SUSIE Raglan Dress s0 1251087D-8716 ALT 1, Model Grup Kodu: EBB, Model Grup Adı: Baskılı Elbise, Model Group Name: Printed Dress, Sezon: H&M S 0, Müşteri: H&M, Marka: H&M, Departman: Young Girl (8716), Tedarik Grubu: 10-204, Ülke Kodu: GB, Sipariş Adedi: 1256, Döviz Türü: EUR, Döviz Birim Fiyat: 4.16, Döviz Ciro: 5224.96, EUR Ciro: 5224.96, Etüt Makina Dk: 10.344, Toplam Etüt Makina Dk: 12992.064, Etüt Dk: 11.604, Toplam Etüt Dk: 14574.624, MA_WorkOr